In [1]:
#import cell for libraries
import numpy as np
import os
import glob
import pathlib
from PIL import Image
from tqdm import tqdm
import pandas as pd
import seaborn as sns
import sklearn
import pickle
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

from matplotlib.colors import LogNorm, Normalize

from IPython.display import display, HTML
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

#tensorflow
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import load_model

In [2]:
def get_path(Id,df):
    """Input: row number of the dataframe
       Returns the filepath for the corresponding image"""
    
    #get path from dataframe
    path2 = df.loc[Id]["filepaths"]
    
    #return the path
    return path2

def get_image(Id,df,x=224,y=224):
    """Returns image of a given row of the birds dataframe"""
    #get initial path to all data
    path1 = "kaggle/input/inputbirds/"
    
    #call fet_path function
    path2 = get_path(Id,df)
    
    #read image of the path1+path2 and return it
    return Image.open(path1+path2).resize((x,y))

def get_input_NN(Id,df,gray = False):
    """Return a 3D matrix of image ready to be read in the CNN"""
    
    #call fet_image function to read image
    im = get_image(Id,df)
    
    #transform iamge to RGB format
    rgb_im = im.convert('RGB')
    
    if gray:
        g_im = rgb_im.convert('L')
        input_NN = asarray(g_im)
                
    else:
    
        #Empty zeros matrix to be filled with pixel values of the image
        input_NN = np.zeros((3,224,224))

        #fill up the matrix with RGB values at each point
        for x in range(224):
            for y in range(224):
                r, g, b = rgb_im.getpixel((x,y))
                input_NN[0,x,y] = r
                input_NN[1,x,y] = g
                input_NN[2,x,y] = b

    #returns the 3D matrix a s a numpy array
    return input_NN


def show_image(Id,inputs):
    """Print image from Id of image in the input array"""
    im = plt.imshow(inputs[Id].astype('uint8'))



def get_class_dict():
    """Return dictionary of classes of each target"""

    class_dict_df = pd.read_csv("/kaggle/input/inputbirds/class_dict.csv")
    class_dict = {}
    for row in range(len(class_dict_df)):
        class_dict[class_dict_df.iloc[row][1]] = class_dict_df.iloc[row][0]
    return class_dict



def CNNDefinition(fil, k_size,  Poolsize, dense, 
                  input_shape, N_categ ,  loss = 'categorical_crossentropy',  
                  metrics = ['accuracy'], last_act = "softmax", lmb = 0.01, eta = 0.001):
    """
    This CNNDefinition function allows the definition of a CNN network. The Network defined can be exploited for both 
    a regression and for a classification task.
    
    Parameters ---
    fil : list of the number of filters per each convolutional layer,
    k_size : dimension of the kernels related to each convolutional layer,
    Poolsize : dimension of the pooling layer applied after each convolutional one,
    dense : list of dimensions of dense layers applied after the convolutional step, 
    input_shape :  dimension of the input data , 
    N_categ : number of categories in the classification task (has to be set at 1 for regression),
    loss = 'categorical_crossentropy',  
    metrics = ['accuracy'], 
    last_act : default = "sigmoid", ok for classification. Has to be changed for regression
    """
    
    model = keras.Sequential()    
    model.add(layers.Conv2D(filters = fil[0], kernel_size = k_size[0],
                            padding = "same",
                            activation = "relu",
                            #strides = 2,
                            input_shape = input_shape,
                            kernel_regularizer=regularizers.l2(lmb)
                            ))
    model.add(layers.MaxPooling2D(Poolsize[0]))
    model.add(layers.BatchNormalization())
    model.add(layers.SpatialDropout2D(0.3))

    for i in range(1,len(fil)):
        model.add(layers.Conv2D(filters = fil[i], kernel_size = k_size[i],
                                padding = "same", activation = "relu",kernel_regularizer=regularizers.l2(lmb)))
        model.add(layers.BatchNormalization())

        model.add(layers.MaxPooling2D(Poolsize[i]))
        model.add(layers.SpatialDropout2D(0.3))


    model.add(layers.Flatten())
    for i in range(len(dense)):
        model.add(layers.Dense(dense[i],activation="relu",kernel_regularizer=regularizers.l2(lmb)))

    #model.add(layers.Dropout(0.2))
    model.add(layers.Dense(N_categ, activation= last_act))

    print(model.summary())

    #default optimizer is Adam, different learning rates affect the time the network takes to converge.
    
    #try different alogrithms and learning rates
    optimizer = keras.optimizers.Adam()
    #optimizer = keras.optimizers.SGD(lr=eta)
    
    optimizer.learning_rate.assign(eta)

    model.compile(
        optimizer = optimizer, 
        loss = loss, 
        metrics = metrics
    )

    return model



def Fit(model, X_train, Y_train, X_test, Y_test, pat = 100, mindelta = 0.01, batch = 64, epochs = 1200, verb = 1):
    
    '''
    Fit function for fitting the model. Training and test data are specified as parameters; 
    EarlyStopping is introduced to avoid overfitting.
    The function returns a dataframe containing the history of the training. The values of the accuracy and the different 
    epochs can be then inferred from such a returned quantity.
    
    Parameters ---
    model : model to be fit,
    X_train : train features,
    Y_train : train targets,
    X_test : test features,
    Y_test : test features,
    pat : default = 100, patience for the EarlyStopping, 
    mindelta : default = 0.01, mindelta in EarlyStopping, 
    batch : default = 64, batch size while fitting, 
    epochs : default = 1200, number of epochs for training,  
    verb = 1):
    '''

    early_stopping = keras.callbacks.EarlyStopping(
        patience = pat,
        min_delta = mindelta,
        restore_best_weights=True,
    )

    history = model.fit(
        X_train, Y_train,
        validation_data=(X_test, Y_test),
        batch_size = batch,
        epochs = epochs,
        callbacks=[early_stopping],
        verbose=verb, # hide the output because we have so many epochs
    )
    return pd.DataFrame(history.history)


def Fit_automatic(model, train, test, pat = 100, mindelta = 0.01, batch = 64, epochs = 1200, verb = 1):
    
    early_stopping = keras.callbacks.EarlyStopping(
        patience = pat,
        min_delta = mindelta,
        restore_best_weights=True,
    )

    history = model.fit(
        train,
        validation_data=test,
        batch_size = batch,
        epochs = epochs,
        callbacks=[early_stopping],
        verbose=verb, # hide the output because we have so many epochs
    )
    return pd.DataFrame(history.history)

### Load File

In [3]:
# inputs = pickle.load(open('../input/inputbirds/inputs_56_56.pkl', 'rb'))

In [4]:
# label_list = list(birds.labels.unique())
# sub_label_list = label_list[0:20]
# birds_20 = birds[birds["labels"].isin(sub_label_list)]

# train_20 = birds_20[birds_20["data set"].isin(['train'])]
# train_20

# inputs = np.zeros((len(train_20),224,224,3))
# key_list = list(train_20.index)
# n = 0
# for Id in tqdm(key_list):
#     try:
#         input_NN = get_input_NN(Id,train_20)
#         inputs[n] = input_NN.T
#     except: 
#         continue
#     n+=1
    
# a_file = open("inputs.pkl", "wb")
# pickle.dump(inputs, a_file)
# a_file.close()

In [5]:
# birds = pd.read_csv("/kaggle/input/inputbirds/birds.csv")
# label_list = list(birds.labels.unique())
# sub_label_list = label_list[0:20]
# birds_20 = birds[birds["labels"].isin(sub_label_list)]

# train_20 = birds_20[birds_20["data set"] == "train"]

# class_dict = get_class_dict()
# labels = np.zeros((2987))
# for label in range(len(train_20)):
#     labels[label] = class_dict[train_20.iloc[label][1]]
    
# inputs = pickle.load(open("/kaggle/input/inputbirds/inputs_56_56.pkl", "rb"))
# inputs = inputs/255

# train_size = 0.8
# test_size = 1 - train_size
# X_train, X_test, Y_train, Y_test = train_test_split(inputs, labels,  train_size=train_size,
#                                                     test_size=test_size)

# Y_train = to_categorical(Y_train)
# Y_test = to_categorical(Y_test)
# input_shape = inputs[0].shape

In [6]:
# l_fil = [20,20]
# l_ker = [[4,4], [3,3]]
# l_pool = [3,2]
# l_dense = []

# # model = CNNDefinition(l_fil, l_ker, l_pool, l_dense, (56, 56, 3), 20)

In [7]:
# history = Fit(model, X_train, Y_train, X_test, Y_test, epochs = 10)

In [8]:
# datagen_vgg = ImageDataGenerator() 
# #
# train_generator_vgg = datagen_vgg.flow_from_directory('/kaggle/input/100-bird-species/train',
#                                                       batch_size=128,
#                                                       target_size=(224,224),
#                                                       class_mode='categorical')
# #
# validation_generator_vgg = datagen_vgg.flow_from_directory('/kaggle/input/100-bird-species/valid',
#                                                            batch_size=128,
#                                                            target_size=(224,224),
#                                                            class_mode='categorical')


In [9]:
# history = Fit_automatic(model, train_generator_vgg, validation_generator_vgg, epochs = 400, batch = 128)

In [10]:
# plt.plot(history.accuracy)
# plt.plot(history.val_accuracy)
# plt.show()

In [11]:
# datagen_vgg = ImageDataGenerator() 
# #
# train_generator_vgg = datagen_vgg.flow_from_directory('/kaggle/input/100-bird-species/train',
#                                                       batch_size=256,
#                                                       target_size=(224,224),
#                                                       class_mode='categorical')
# #
# validation_generator_vgg = datagen_vgg.flow_from_directory('/kaggle/input/100-bird-species/valid',
#                                                            batch_size=256,
#                                                            target_size=(224,224),
#                                                            class_mode='categorical')

# l_fil = [20,10,]
# l_ker = [[4,4], [3,3]]
# l_pool = [3,4,]
# l_dense = []

# model = CNNDefinition(l_fil, l_ker, l_pool, l_dense, data_pca.shape[1:4], Y_train.shape[1],)


# history = Fit_automatic(model_total, train = train_generator_vgg,
#                             epochs=200,
#                             batch = 256,
#                             test=validation_generator_vgg,
#                             )

In [12]:
train="../input/100-bird-species/train"
test="../input/100-bird-species/test"
valid="../input/100-bird-species/valid"


def process(data):
    path=pathlib.Path(data)#converting the string to path
    filepaths=list(path.glob(r"*/*.jpg"))#Going through all the subpaths 
    labels=list(map(lambda x: os.path.split(os.path.split(x)[0])[1],filepaths))#Separating the label from filepath and storing it
    df1=pd.Series(filepaths,name='filepaths').astype(str)
    df2=pd.Series(labels,name='labels')
    df=pd.concat([df1,df2],axis=1)#Making the dataframe
    return df



df_train=process(train)
df_test=process(test)
df_valid=process(valid)

In [13]:
#datagen = ImageDataGenerator(preprocessing_function=preprocess_input, zoom_range = (0.8, 1.2)) 
##
#train_generator = datagen.flow_from_directory('/kaggle/input/100-bird-species/train',
#                                                      batch_size=256,
#                                                      target_size=(224,224),
#                                                      class_mode='categorical')
##
#validation_generator = datagen.flow_from_directory('/kaggle/input/100-bird-species/valid',
#                                                           batch_size=256,
#                                                           target_size=(224,224),
#                                                           class_mode='categorical')
#

train_generator=ImageDataGenerator( preprocessing_function=preprocess_input)
test_generator=ImageDataGenerator( preprocessing_function=preprocess_input)
valid_generator=ImageDataGenerator( preprocessing_function=preprocess_input)

train_image=train_generator.flow_from_dataframe(dataframe=df_train,
                                                x_col='filepaths',
                                                y_col='labels',
                                                target_size=(224,224),
                                                batch_size=16,
                                                subset='training',
                                                random_seed=42)

test_image = test_generator.flow_from_dataframe(
    dataframe=df_test,
    x_col='filepaths',
    y_col='labels',
    target_size=(224,224),
    batch_size=32
)

valid_image = test_generator.flow_from_dataframe(
    dataframe=df_valid,
    x_col='filepaths',
    y_col='labels',
    subset='training',
    target_size=(224,224),
    batch_size=32)

Found 54652 validated image filenames belonging to 375 classes.
Found 1875 validated image filenames belonging to 375 classes.
Found 1875 validated image filenames belonging to 375 classes.


In [14]:
model_VGG = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
model_VGG.trainable = False

#model_VGG.summary()
flattening_layer = layers.Flatten()
dout_layer = layers.Dropout(0.4)
bnor_layer = layers.BatchNormalization()
last_layer = layers.Dense(375, activation='softmax')

model_total = Sequential([model_VGG, flattening_layer, dout_layer, last_layer])

model_total.summary()

model_total.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics="accuracy")

2022-02-12 22:20:59.868187: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-12 22:21:00.056606: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-12 22:21:00.057500: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-12 22:21:00.059260: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

58900480/58889256 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dropout (Dropout)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 375)               9408375   
Total params: 24,123,063
Trainable params: 9,408,375
Non-trainable params: 14,714,688
_________________________________________________________________


In [15]:
history_VGG = Fit_automatic(model_total, train = train_image,
                            epochs = 15,
                            batch = 64,
                            test = valid_image,
                            pat = 15
                            )
history_VGG.to_csv("Output - VGG 0.4Dout - BNorm.csv")

2022-02-12 22:21:04.620167: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/15


2022-02-12 22:21:06.185835: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


3416/3416 [==============================] - 394s 113ms/step - loss: 69.1321 - accuracy: 0.5713 - val_loss: 33.4131 - val_accuracy: 0.8224
Epoch 2/15
3416/3416 [==============================] - 174s 51ms/step - loss: 22.6525 - accuracy: 0.8696 - val_loss: 34.6616 - val_accuracy: 0.8491
Epoch 3/15
3416/3416 [==============================] - 175s 51ms/step - loss: 16.3215 - accuracy: 0.9105 - val_loss: 38.3717 - val_accuracy: 0.8533
Epoch 4/15
3416/3416 [==============================] - 177s 52ms/step - loss: 12.5071 - accuracy: 0.9333 - val_loss: 38.8952 - val_accuracy: 0.8741
Epoch 5/15
3416/3416 [==============================] - 179s 52ms/step - loss: 10.6516 - accuracy: 0.9457 - val_loss: 39.6652 - val_accuracy: 0.8779
Epoch 6/15
3416/3416 [==============================] - 179s 52ms/step - loss: 8.0819 - accuracy: 0.9580 - val_loss: 41.7413 - val_accuracy: 0.8837
Epoch 7/15
3416/3416 [==============================] - 177s 52ms/step - loss: 7.7663 - accuracy: 0.9613 - val_loss: 

In [16]:
# fig = plt.figure(figsize=(20,15))
# gs = fig.add_gridspec(4, 4)
# #
# for row in range(0, 3):
#     for col in range(0, 3):
#         image, label = train_generator_vgg.next()
#         label_id = list(label[0]).index(1)
#         label_str = list(train_generator_vgg.class_indices.keys())[list(train_generator_vgg.class_indices.values()).index(label_id)]
#         ax = fig.add_subplot(gs[row, col])
#         ax.axis('off');
#         ax.set_title(label_str, fontdict={"fontweight": 700})
#         ax.imshow(image[0].astype('uint8'));
        

In [17]:
from tensorflow.keras.applications.efficientnet import EfficientNetB4 

model_ENet = EfficientNetB4(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
model_ENet.trainable = False

#model_ENet.summary()
flattening_layer = layers.Flatten()
bnor_layer2 = layers.BatchNormalization()
dout_layer = layers.Dropout(0.4)
dense_layer_1 = layers.Dense(4096)
dense_layer_2 = layers.Dense(4096)
last_layer = layers.Dense(375, activation='softmax')

model_total_ENet = Sequential([model_ENet, flattening_layer, bnor_layer2, dout_layer, last_layer])
model_total_ENet.summary()

model_total_ENet.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics="accuracy")

71696384/71686520 [==============================] - 1s 0us/step
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb4 (Functional)  (None, 7, 7, 1792)        17673823  
_________________________________________________________________
flatten_1 (Flatten)          (None, 87808)             0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 87808)             351232    
_________________________________________________________________
dropout_1 (Dropout)          (None, 87808)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 375)               32928375  
Total params: 50,953,430
Trainable params: 33,103,991
Non-trainable params: 17,849,439
_________________________________________________________________


In [18]:
history_ENet = Fit_automatic(model_total_ENet, train = train_image,
                            epochs=15,
                            batch = 256,
                            test=valid_image,
                            pat = 15, mindelta = 0.01
                            )
history_ENet.to_csv("Output - preprocessing ENet - Dout 0.4  - 4096 x 2 -  0.6Dout.csv")
model_total_ENet.save('model_total_ENet.h5')

Epoch 1/15
3416/3416 [==============================] - 237s 67ms/step - loss: 20.3562 - accuracy: 0.5776 - val_loss: 7.9975 - val_accuracy: 0.8347
Epoch 2/15
3416/3416 [==============================] - 231s 68ms/step - loss: 7.1275 - accuracy: 0.8573 - val_loss: 8.7202 - val_accuracy: 0.8464
Epoch 3/15
3416/3416 [==============================] - 225s 66ms/step - loss: 5.3095 - accuracy: 0.8976 - val_loss: 8.7717 - val_accuracy: 0.8736
Epoch 4/15
3416/3416 [==============================] - 225s 66ms/step - loss: 4.1258 - accuracy: 0.9229 - val_loss: 8.6091 - val_accuracy: 0.8784
Epoch 5/15
3416/3416 [==============================] - 234s 68ms/step - loss: 3.3271 - accuracy: 0.9378 - val_loss: 8.9278 - val_accuracy: 0.8811
Epoch 6/15
3416/3416 [==============================] - 223s 65ms/step - loss: 2.6788 - accuracy: 0.9501 - val_loss: 8.7686 - val_accuracy: 0.8955
Epoch 7/15
3416/3416 [==============================] - 223s 65ms/step - loss: 2.3099 - accuracy: 0.9591 - val_loss: 

/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [19]:
dense_layer_1_vgg = layers.Dense(4096)
dense_layer_2_vgg = layers.Dense(4096)
last_layer = layers.Dense(375, activation='softmax')


model_total_2 = Sequential([model_VGG, flattening_layer, dout_layer, dense_layer_1_vgg,  dense_layer_2_vgg, last_layer])

model_total_2.summary()

model_total_2.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics="accuracy")

history_VGG = Fit_automatic(model_total_2, train = train_image,
                            epochs = 15,
                            batch = 64,
                            test = valid_image,
                            pat = 15
                            )
history_VGG.to_csv("Output - VGG flat-  dout 04 - 4096 x 2.csv")
model_total_2.save('modeltotal2.h5')

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          multiple                  0         
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
dense_4 (Dense)              (None, 4096)              102764544 
_________________________________________________________________
dense_5 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dense_6 (Dense)              (None, 375)               1536375   
Total params: 135,796,919
Trainable params: 121,082,231
Non-trainable params: 14,714,688
_______________________________

In [20]:
dense_layer_1 = layers.Dense(4096)
dense_layer_2 = layers.Dense(4096)
last_layer = layers.Dense(375, activation='softmax')

model_total_ENet_2 = Sequential([model_ENet, flattening_layer,  dout_layer, dense_layer_1, dense_layer_2, last_layer])
model_total_ENet_2.summary()

model_total_ENet_2.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics="accuracy")

history_ENet = Fit_automatic(model_total_ENet, train = train_image,
                            epochs=15,
                            batch = 64,
                            test=valid_image,
                            pat = 15, mindelta = 0.01
                            )
history_ENet.to_csv("Output - preprocessing ENet - Dout 0.4  - 4096 x 2 -  0.6Dout.csv")
model_total_ENet_2.save("model_total_ENet_2.h5")

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb4 (Functional)  (None, 7, 7, 1792)        17673823  
_________________________________________________________________
flatten_1 (Flatten)          multiple                  0         
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
dense_7 (Dense)              (None, 4096)              359665664 
_________________________________________________________________
dense_8 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dense_9 (Dense)              (None, 375)               1536375   
Total params: 395,657,174
Trainable params: 377,983,351
Non-trainable params: 17,673,823
_______________________________

2022-02-13 01:58:06.857157: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1438646272 exceeds 10% of free system memory.


## Try to use the TPUs
